In [1]:
# Imports
from openai import OpenAI

import os

from utils import constants
from utils import util
from utils import keys
from utils import prompts

In [2]:
# Load data and Initialize OpenAI
os.environ['OPENAI_API_KEY'] = keys.WX
client = OpenAI()
client.api_key = os.getenv('OPENAI_API_KEY')

dataset, datakey = util.load_data()
# wp_all = dataset[0]
sp_test = dataset[1]
sp_all = dataset[2]
# wp_test = dataset[3]
# wp_base, wp_sr, wp_cr = util.split_data_by_type(wp_all)
# sp_base, sp_sr, sp_cr = util.split_data_by_type(sp_all)

Loaded WP-train.npy at index 0
Loaded SP_eval_data_for_practice.npy at index 1
Loaded SP-train.npy at index 2
Loaded WP_eval_data_for_practice.npy at index 3


In [3]:
# Model settings
MODEL = "gpt-3.5-turbo"
SYSTEM_PROMPT = {"role": "system", 
                 "content": "You are a Question Answering Model, your response must be a number from the choices that are delimited by the symbol \";\" ."}
MULTI_PREFIX = "You are a Question Answering Model, your response must be a number from the choices that are delimited by the symbol \";\". Here are some examples: \n"
SP_QUESTION = "Think outside of the box and respond with the number corresponding to the best choice for the following question.\nQuestion: "
WP_QUESTION = "For the following word problem, look at the meaning and letters in the words and respond with the number corresponding to the best choice.\nQuestion: "

In [7]:
# Eval Multishot GPT3.5-TURBO
preds = util.run_eval_cot(client, MODEL, sp_test, prompts.CHAIN_SP_BASE, prompts.CHAIN_SYSTEM, MULTI_PREFIX, m=0, n=len(sp_test))

In [8]:
util.submission_log(preds, "answer_sen")

Saved submission to /Users/alvinchen/Documents/GitHub/brainteaser-data/submission/answer_sen.txt


In [98]:
import re

with open("test_logs/gpt-3.5-turbo_eval_2023-12-18_21-48-05.log", "r") as f:
    raw = f.readlines()
lines = raw[30:]

In [99]:
# Copy answers and ids
ans = {}
for i in range(len(lines)-1):
    m = re.search(r"Question [0-9]+:", lines[i])
    if m is not None:
        n = m.group(0)
        id = int(re.search(r"[0-9]+", n).group(0))
        an = re.search(r"answer is [0-3]", lines[i+1])
        if an is not None:
            an = int(an.group(0)[-1])
            ans[id] = an

In [100]:
notin = []
for i in range(1, 120):
    if i not in ans:
        notin.append(i)
print(notin)

[3, 23, 26, 28, 31, 34, 38, 58, 59, 92, 95, 114]


In [102]:
repeatid = []
test_questions = []
order = []
for data in sp_test:
    test_questions.append(data["question"])
for data in sp_all:
    if data["question"] in test_questions:
        repeatid.append(data["id"])
        m = [i for i in range(len(test_questions)) if test_questions[i] == data["question"]][0]
        order.append(m)
print(order)

[38, 47, 98, 39, 5, 23, 3, 102, 51, 28, 50, 77, 1, 24, 88, 94, 71, 85, 11, 111, 93, 86, 119, 66, 104, 108, 61, 60, 116, 73, 59, 110, 52, 72, 36, 17, 92, 21, 69, 62, 113, 107, 90, 81, 112, 35, 16, 57, 19, 40, 43, 70, 46, 79, 13, 83, 64, 18, 49, 27, 58, 115, 31, 33, 2, 109, 6, 9, 8, 97, 84, 41, 89, 10, 101, 91, 53, 106, 55, 117, 82, 99, 37, 56, 87, 105, 44, 100, 67, 22, 20, 15, 12, 103, 75, 4, 7, 14, 29, 34, 32, 42, 65, 95, 25, 96, 114, 30, 54, 118, 63, 76, 0, 26, 74, 78, 45, 80, 68, 48]


In [104]:
correct = []
label = []
choices = []
for i in order:
    id = repeatid[i]
    for data in sp_all:
        if data["id"] == id:
            correct.append(data["answer"])
            label.append(data["label"])
            choices.append("".join(str(j) + " = " + data["choice_list"][j] + "; " for j in range(4)))

In [105]:
with open("../submission/answer_sen.txt", "r") as f:
    answers = f.readlines()

with open("chain_of_thought_output.txt", "w") as f:
    for r in raw[:29]:
        f.write(r)
    for i in range(120):
        f.write(lines[i*3])
        f.write("Choices: " + choices[i] + "\n")
        f.write(lines[i*3+1])
        answer = answers[i].strip()
        f.write(f"Corresponding Answer: {str(answer)} = {sp_test[i]['choice_list'][int(answers[i])]}\n")
        f.write(f"Correct Answer: {label[i]} = {correct[i]}\n")
        f.write("\n")